# Subway Data Preprocessing

**Data Source:**

  ./network: Subway network metadata (created manually for this project)

  ./ridership_daily: https://data.seoul.go.kr/dataList/OA-12914/S/1/datasetView.do

  ./ridership_hourly: https://data.seoul.go.kr/dataList/OA-12921/F/1/datasetView.do
  

**Input:**

  ./network/lines.csv

  ./network/station_metadata.csv

  ./network/station_to_id.csv

  ./ridership_daily/*.csv

  ./ridership_hourly/*.csv 


**Output:**

  network_nodes.json

  network_adjacency.npy

  ridership_daily.json
  
  ridership_hourly.json


**Helper functions**

In [1]:
import csv

def csv_to_list(fpath: str, delim: str = ','):
  try:
    with open(fpath, "r", encoding="euc-kr") as f:
      return list(csv.reader(f, delimiter=delim))
  except:
    with open(fpath, "r", encoding="utf-8") as f:
      return list(csv.reader(f, delimiter=delim))

**Preprocess subway network data (nodes & adjacency)**

In [2]:
import numpy as np
import geopandas as gpd

rows = csv_to_list('./network/station_metadata.csv', ';')
N = len(rows) - 1
adjacency = np.zeros((N, N))
nodes = {
  'station_id': [None] * N, 
  'station_name_ENG': [None] * N, 
  'lat': [None] * N, 
  'lng': [None] * N
}
for i, (st_id, st_ENG, adjacent_ids, lat, lng) in enumerate(rows[1:]):
  st_id = int(st_id)
  # set node (station) information
  if (i != st_id
      or nodes['station_id'][st_id] != None 
      or nodes['station_name_ENG'][st_id] != None 
      or nodes['lat'][st_id] != None 
      or nodes['lng'][st_id] != None):
    raise ValueError(f'Invalid station id: {st_id} ({st_ENG})')
  nodes['station_id'][st_id] = st_id
  nodes['station_name_ENG'][st_id] = st_ENG 
  nodes['lat'][st_id] = float(lat) 
  nodes['lng'][st_id] = float(lng)
  # update adjacency matrix
  for adj_id in adjacent_ids.split(','):
    adj_id = int(adj_id)
    adjacency[st_id, adj_id] = adjacency[adj_id, st_id] = 1
nodes = gpd.GeoDataFrame(
  data=nodes, 
  geometry=gpd.points_from_xy(x=nodes['lng'], y=nodes['lat']), 
  crs='EPSG:4326'
)
nodes[['station_id', 'station_name_ENG', 'lat', 'lng']].to_json('network_nodes.json')
np.save('network_adjacency.npy', adjacency)
print(adjacency.shape)

(521, 521)


In [3]:
nodes.head(5)

,station_id,station_name_ENG,lat,lng,geometry
0,0,April 19th National Cemetery station,37.649504,127.013688,POINT (127.01369 37.64950)
1,1,Ganeung station,37.748333,127.044167,POINT (127.04417 37.74833)
2,2,Garak Market station,37.493056,127.118056,POINT (127.11806 37.49306)
3,3,Gasan Digital Complex station,37.480700,126.882950,POINT (126.88295 37.48070)
4,4,Gayang station,37.561111,126.855278,POINT (126.85528 37.56111)


In [4]:
from cartoframes.viz import Layer
Layer(nodes)

**Preprocess daily ridership data**

In [5]:
import glob
import pandas as pd
import math
from collections import OrderedDict

def str_to_int(s: str):
  d, i = math.modf(float(s))
  assert d == 0
  return int(i)

def get_station_to_id(path: str = './network/station_to_id.csv'):
  st_to_id = dict()
  rows = csv_to_list(path, ';')
  for st_id, st_KOR, line_id in rows[1:]:
    key = (st_KOR, int(line_id))
    assert not key in st_to_id
    st_to_id[key] = int(st_id)
  return st_to_id

def get_line_to_id(path: str = './network/lines.csv'):
  d = dict()
  rows = csv_to_list(path, ';')
  for line_id, line_KOR, _ in rows[1:]:
    d[line_KOR] = int(line_id)
  return d
  
station_to_id = get_station_to_id('./network/station_to_id.csv')
line_to_id = get_line_to_id('./network/lines.csv')

def get_daily_ridership():
  data = {
    'IN': [],
    'OUT': []
  }
  keys = OrderedDict()
  for fpath in glob.glob('./ridership_daily/*.csv'):
    rows = csv_to_list(fpath)
    for row in rows[1:]:
      date, line, station, passengers_IN, passengers_OUT = row[:5]
      station_id: int = station_to_id[(station, line_to_id[line])]
      passengers_IN = float(passengers_IN)
      passengers_OUT = float(passengers_OUT)
      key = (date, station_id)
      if key in keys:
        data['IN'][keys[key]] += passengers_IN
        data['OUT'][keys[key]] += passengers_OUT
      else:
        keys[key] = len(data['IN'])
        data['IN'].append(passengers_IN)
        data['OUT'].append(passengers_OUT)
  data = pd.DataFrame(data, index=pd.MultiIndex.from_tuples(keys.keys(), names=['date', 'station_id']))
  data.index = data.index.set_levels(pd.to_datetime(data.index.levels[0]).date, level=0)
  data.sort_index(inplace=True)
  return data
  
df = get_daily_ridership()
df.reset_index(inplace=True, drop=False)
df.to_feather('ridership_daily.ftr')
print(df.shape)
df.head(5)

(849400, 4)


,date,station_id,IN,OUT
0,2017-01-01,1,4194.0,3852.0
1,2017-01-01,2,6572.0,7062.0
2,2017-01-01,3,18698.0,19945.0
3,2017-01-01,4,11004.0,10890.0
4,2017-01-01,6,2227.0,2222.0


**Preprocess hourly ridership data**

In [6]:
import glob
import pandas as pd
from collections import OrderedDict
from tqdm import notebook

header_map = {
  '05~06': '~06',
  '06시이전': '~06',
  '06이전': '~06',
  '24시이후': '24~',
  '24이후': '24~',
  '00~01': '24~',
  '구분': 'in/out',
  '날짜': 'date',
  '역명': 'station_name_KOR',
  '역번호': 'station_id',
  '합계': 'total', 
  '호선': 'line'
}

in_out_map = {'승차': 'in', '하차': 'out'}

selected_columns = ['date', 'station_id', 'in/out', '~06', '06~07',
  '07~08', '08~09', '09~10', '10~11', '11~12', '12~13', '13~14', '14~15',
  '15~16', '16~17', '17~18', '18~19', '19~20', '20~21', '21~22', '22~23',
  '23~24', '24~']

def get_old_to_new_id():
  old_to_new_id = dict()
  rows = csv_to_list('./ridership_hourly/서울교통공사 2020년 일별 시간대별 역별 승하차 인원(1_8호선).csv', ',')
  for row in rows[1:]:
    line, old_id, st_KOR = row[1:4]
    old_id = int(old_id)
    new_id = station_to_id[(st_KOR, line_to_id[line])]
    if old_id in old_to_new_id:
      assert old_to_new_id[old_id] == new_id
    else:
      old_to_new_id[old_id] = new_id
  return old_to_new_id

def fix_header(header: list):
  def fix_elem(e: str):
    e = e.replace(' ', '').replace(':00', '')
    return header_map[e] if e in header_map else e
    
  return [fix_elem(elem) for elem in header]

old_to_new_id = get_old_to_new_id()

def fix_column(col: pd.Series, t: str, station_id: bool):
  def fix_str(s: str):
    s = s.replace(' ', '')
    return in_out_map[s] if s in in_out_map else s

  if t == 'str':
    l = [fix_str(e) for e in col]
  elif t == 'int':
    l = [int(e.replace(',', '')) for e in col]
    if station_id:
      for i, old_id in enumerate(l):
        l[i] = old_to_new_id[old_id]
  return l

def read_df(path: str):
  rows = csv_to_list(path)
  df = pd.DataFrame(rows[1:], columns=fix_header(rows[0]), copy=False)
  df = df[selected_columns]
  df = df.loc[:,~df.columns.duplicated(keep='last')]
  for col in df.columns.values:
    df[col] = fix_column(df[col], 'str' if col in {'date', 'in/out'} else 'int', col == 'station_id')
  return df

def get_df(path: str):
  df = read_df(path)
  keys = OrderedDict()
  data = {
    'IN': [],
    'OUT': []
  }
  for row in df.itertuples(index=False):
    date, st_id, in_out = row[:3]
    for ts, val in [('00~06', row[3]), ('06~10', sum(row[4:8])), ('10~16', sum(row[8:14])), ('16~21', sum(row[14:19])), ('21~00', sum(row[19:]))]:
      key = (date, ts, st_id)
      if key in keys:
        if in_out == 'in':
          data['IN'][keys[key]] += val
        else:
          data['OUT'][keys[key]] += val
      else:
        keys[key] = len(data['IN'])
        if in_out == 'in':
          data['IN'].append(val)
          data['OUT'].append(0)
        else:
          data['IN'].append(0)
          data['OUT'].append(val)
  data = pd.DataFrame(data, index=pd.MultiIndex.from_tuples(keys.keys(), names=['date', 'time', 'station_id']))
  data.index = data.index.set_levels(pd.to_datetime(data.index.levels[0]).date, level=0)
  return data

df = pd.concat([get_df(f_path) for f_path in notebook.tqdm(glob.glob('./ridership_hourly/*.csv'))], ignore_index=False, copy=False)
df.sort_index(inplace=True)
df.reset_index(inplace=True, drop=False)
print(df.shape)
df.to_feather('ridership_hourly.ftr')
df.head(5)

  0%|          | 0/4 [00:00<?, ?it/s]

(1762160, 5)


,date,time,station_id,IN,OUT
0,2017-01-01,00~06,2,93,24
1,2017-01-01,00~06,3,92,38
2,2017-01-01,00~06,11,1117,148
3,2017-01-01,00~06,12,86,33
4,2017-01-01,00~06,13,182,25
